In [16]:
#Required Libraries
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from Country_and_Medals_Lists import medals_list 
import re 

In [19]:
# Required Header
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

#Initialize dataframe
countrylist = pd.DataFrame(columns=["Olympic Session", "Country Order"])

# Load the csv's rows
urls = pd.read_csv("past_olympics.csv")
 
cities = urls["City"].iloc[40:].tolist()
print(cities)
# Create the full url list
first_part = "https://olympics.com/en/olympic-games/"
last_part = "/medals"
path = []

for city in cities:
    full_url = f'{first_part}{city}{last_part}'
    path.append(full_url)

# Class
    scraper = medals_list()

# Initialize extracting
    for site in path:
        responses = requests.get(site, headers=header)
        soup = bs(responses.content, "html.parser")
    # Call functions individually
        names = scraper.extract_name(soup)  
        country_codes, country_names = scraper.extract_country_list(soup)
        scraper.extract_medals(soup)

prz = scraper.prizes
#prz

prz.to_csv("Olympic Prizes fifth Part.csv", index = False)
print("Successfull saved")
    

['berlin-1936', 'garmisch-partenkirchen-1936', 'los-angeles-1932', 'lake-placid-1932', 'amsterdam-1928', 'st-moritz-1928', 'paris-1924', 'chamonix-1924', 'antwerp-1920', 'stockholm-1912', 'london-1908', 'st-louis-1904', 'paris-1900', 'athens-1896']
Successfull saved


In [20]:
country_table = scraper.country_df
country_table.to_csv("Olympic Countries fifth part.csv", index= False)

Olympic Games Paris 1900 and Olympic Games St. Louis 1904 didn't work with the created class for Olympic site, so was necessary research another resource, and save it in the existing document.

In [ ]:

urls = [
    ("https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Par%C3%ADs_1900", "Olympic Games Paris 1900"),
    ("https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_San_Luis_1904", "Olympic Games St. Louis 1904")
]

# Función para limpiar los nombres de países y extraer lo deseado
def clean_country(country):
    match = re.search(r'\((.*?)\)', country)  # Busca lo que está entre paréntesis
    return match.group(1) if match else country  # Devuelve el texto entre paréntesis o el original

# Lista para almacenar los resultados
all_data = []

for url, event in urls:
    # Leer las tablas de la página
    tables = pd.read_html(url)
    
    # Seleccionar la tabla deseada (basado en índice o contenido, aquí asumimos la primera)
    table = tables[0]
    
    # Renombrar columnas
    table.rename(columns={"Núm.": "Rank",
                    "País": "Country Code",
                    "Unnamed: 2": "Gold",
                    "Unnamed: 3": "Silver",
                    "Unnamed: 4": "Bronze"}, inplace=True)

    # Limpiar la columna 'Country'
    table['Country Code'] = table['Country Code'].apply(clean_country)

    # Eliminar filas donde 'Country Code' sea 'Total'
    table = table[table['Country Code'] != "Total"].copy()
    
    # Añadir columna sin generar advertencias
    table['Event Name'] = event
    
    # Seleccionar columnas deseadas
    table = table[["Event Name", "Country Code", "Gold", "Silver", "Bronze"]]
    
    # Agregar los datos a la lista
    all_data.append(table)

# Combinar todos los datos en un único DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Mostrar el DataFrame final
print(final_df)

# Guardar el DataFrame en el archivo existente (añadir datos al final si existe)
try:
    existing_df = pd.read_csv("Olympic Prizes.csv")  # Leer el archivo existente
    combined_df = pd.concat([existing_df, final_df], ignore_index=True)  # Combinar los datos
except FileNotFoundError:
    combined_df = final_df  # Si no existe, usar solo el nuevo DataFrame

print(len(existing_df))
# Guardar el DataFrame combinado en el archivo
combined_df.to_csv("Olympic Prizes.csv", index=False)
print(len(combined_df))

                      Event Name Country Code  Gold  Silver  Bronze
0       Olympic Games Paris 1900          FRA    26      41      34
1       Olympic Games Paris 1900          USA    19      14      14
2       Olympic Games Paris 1900          GBR    15       6       9
3       Olympic Games Paris 1900          ZZX     6       3       3
4       Olympic Games Paris 1900          SUI     6       2       1
5       Olympic Games Paris 1900          BEL     5       5       5
6       Olympic Games Paris 1900          GER     4       2       2
7       Olympic Games Paris 1900          ITA     2       2       0
8       Olympic Games Paris 1900          AUS     2       0       3
9       Olympic Games Paris 1900          DEN     1       3       2
10      Olympic Games Paris 1900          HUN     1       2       2
11      Olympic Games Paris 1900          CUB     1       1       0
12      Olympic Games Paris 1900          CAN     1       0       1
13      Olympic Games Paris 1900          ESP   